# Game of Life - the problem of self-replicating systems with cellular automation
### created by John Horton Conway in 1970

This is a zero player game, where the evolution is determined by some initial conditions and you get to observe how it evolves.
It is 'Turing Complete' it can simulate a universal constructor or any other turing machine
 
The world is an infinite grid of square cells each of which could be in two possible states live or dead (or if you wish populated and unpopulated)
Every cell interacts with its eights neighbors, horizontally, vertically, and diagonally

There are only 4 rules that govern everything-

Any live cell with fewer than two live neighbors dies, as if by underpopulation.
Any live cell with two or three live neighbors lives on to the next generation.
Any live cell with more than three live neighbors dies, as if by overpopulation.
Any dead cell with exactly three live neighbors becomes a live cell, as if by reproduction.

the initial pattern constituters the seed of the system, first generation is created by applying the above rules simultaneously to every cell in the seed, live or dead, births and deaths occur simultaneously, during the discrete moement called a tick. The rules continue to be applied repeatedly to create further generations :) 

Before looking at my implementation below, I encourage you to program your own implementation, throughout the process you'll learn so much.

It goes without saying that my world is not infiinite - I've made it so my boundary cells (outside_boundary variable) can either be all dead or all alive. When I first learned about this automation I was fasicnated and amazed on how such an interesting and dyamic system could be created from just four simple rules. I love the simplicity and beauty, I hope you will too. 


In [ ]:
import numpy as np
import time

tick_delay_main = 2 # our tick delay
world_width_main, world_height_main = 20, 20 # we'll keep our world limited for, 
tick_life_main = 100
outside_boundary = 0 # cells outside boundary considered what

def apply_rules(cell, neighbors):
    live_neighbors = neighbors.count(1)
    if cell:  # cell is alive
        if live_neighbors < 2:  #less than 2 live neighbors then dies :(
            return 0
        elif live_neighbors == 2 or live_neighbors == 3:  #2 or 3 live neighbors lives on :)
            return 1
        else:  # >3 live neighbors, dies from over popoulation :(
            return 0
    else:  # cell dead
        if live_neighbors == 3:  #if 3 live neibhors then it becomes alive :)
            return 1
    return 0  # dead cells that don't have 3 live neighbors die

def get_neighbors(cell, world_check):
    #return a lsit of all 8 neighbors any neighbor off the world is classfiied as outside_boundary
    row, col = cell[0], cell[1] #y,x  
    neighbors = [] #list to store the aliveness of our 8 neighbors
    bounding_cells = [[row+r-1,col+c-1] for r in range(3) for c in range(3) if (c*r!=1)] #the c*r!=1 is for that it doesn't count itself (cell) as a bounding
    for bc in bounding_cells:
        if 0<=bc[0]<world_check.shape[0] and 0<=bc[1]<world_check.shape[1]:# make sure we're inside the world :)    
            neighbors.append(world_check[bc[0],bc[1]])
        else:
            neighbors.append(outside_boundary) # cells outside the world boundary (we could random this also) 
    return neighbors

def game_loop(world_view, tick_delay, tick_life ):
    for world_tick in range(tick_life):
        world_buffer=world_view.copy()
        stable=True
        for r in range(world_view.shape[0]):
            for c in range(world_view.shape[1]):
                world_buffer[r,c] = apply_rules(world_view[r,c], get_neighbors((r,c), world_view)) # modify the world buffer according to rules applied on world_view
                stable = (world_buffer[r,c]==world_view[r,c]) and stable # once it goes not stable stability cannot be restored 
        world_view=world_buffer.copy() # page swap :) 
        print(world_view) #print out the world_view
        print(f'tick {world_tick} ** population {np.count_nonzero(world_view)} ** stable {stable}') # show some stats, stable means nothign is changing
        time.sleep(tick_delay)
        
#create an all dead world     
world_view_main = np.zeros((world_height_main, world_width_main), dtype=np.byte)
#create a couple occilators that just keep repeating in place


world_view_main[8,10]=world_view_main[9,10]=world_view_main[10,10]=1 #toad oscilator 
world_view_main[15,15]=world_view_main[15,16]=world_view_main[15,17]=world_view_main[16,14]=world_view_main[16,15]=world_view_main[16,16]=1 #toad oscilator 

game_loop(world_view_main,tick_delay_main,tick_life_main) #let's run our main game loop  

